# Bayesian Optimization for Function Maximization

This notebook implements Bayesian Optimization using Gaussian Process Regression to find maximum values of 8 unknown functions with varying dimensions (2D to 8D).

**Approach:**
- Gaussian Process as surrogate model
- Expected Improvement acquisition function
- Sequential optimization for efficient search

## 1. Setup and Data Loading

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import Matern
from sklearn.preprocessing import MinMaxScaler
from scipy.stats import norm

# Set random seed
SEED = 42
np.random.seed(SEED)

In [ ]:
# Load initial data
print("Loading initial data...\n")

for i in range(1, 9):
    X = np.load(f"initial_data/function_{i}/initial_inputs.npy")
    y = np.load(f"initial_data/function_{i}/initial_outputs.npy")
    print(f"Function {i}: {X.shape[1]}D, {X.shape[0]} samples, range [{y.min():.3f}, {y.max():.3f}]")

## 2. Bayesian Optimizer

In [ ]:
class BayesianOptimizer:
    def __init__(self, function_id, seed=42):
        self.function_id = function_id
        self.seed = seed
        
        # Load data
        self.X_train = np.load(f"initial_data/function_{function_id}/initial_inputs.npy")
        self.y_train = np.load(f"initial_data/function_{function_id}/initial_outputs.npy")
        
        # Normalize inputs
        self.scaler = MinMaxScaler()
        self.X_scaled = self.scaler.fit_transform(self.X_train)
        
        # Standardize outputs
        self.y_mean = np.mean(self.y_train)
        self.y_std = np.std(self.y_train) + 1e-8
        self.y_scaled = (self.y_train - self.y_mean) / self.y_std
        
        # Fit Gaussian Process
        kernel = Matern(length_scale=1.0, nu=2.5)
        self.gp = GaussianProcessRegressor(kernel=kernel, alpha=1e-6, 
                                           n_restarts_optimizer=10, random_state=seed)
        self.gp.fit(self.X_scaled, self.y_scaled)
    
    def expected_improvement(self, X, xi=0.01):
        X_scaled = self.scaler.transform(X)
        mu, sigma = self.gp.predict(X_scaled, return_std=True)
        mu_best = np.max(self.y_scaled)
        
        with np.errstate(divide='warn'):
            imp = mu - mu_best - xi
            Z = imp / sigma
            ei = imp * norm.cdf(Z) + sigma * norm.pdf(Z)
            ei[sigma == 0.0] = 0.0
        return ei
    
    def suggest_next_point(self, n_candidates=10000):
        np.random.seed(self.seed)
        n_dims = self.X_train.shape[1]
        candidates = np.random.uniform(0, 1, (n_candidates, n_dims))
        ei_values = self.expected_improvement(candidates)
        return candidates[np.argmax(ei_values)]
    
    def predict(self, point):
        point = np.atleast_2d(point)
        point_scaled = self.scaler.transform(point)
        mu_scaled, sigma_scaled = self.gp.predict(point_scaled, return_std=True)
        mu = mu_scaled[0] * self.y_std + self.y_mean
        sigma = sigma_scaled[0] * self.y_std
        return mu, sigma
    
    def format_query(self, point):
        return "-".join([f"{x:.6f}" for x in point])

## 3. Run Optimization

In [ ]:
# Optimize all functions
results = {}

print("Running Bayesian Optimization...\n")

for func_id in range(1, 9):
    opt = BayesianOptimizer(func_id, seed=SEED)
    next_point = opt.suggest_next_point()
    mean, std = opt.predict(next_point)
    query = opt.format_query(next_point)
    
    results[func_id] = query
    
    print(f"Function {func_id} ({opt.X_train.shape[1]}D):")
    print(f"  Query: {query}")
    print(f"  Predicted: {mean:.6f} ± {std:.6f}")
    print(f"  Current best: {opt.y_train.max():.6f}\n")

## 4. Submission Queries

In [ ]:
print("Formatted queries for submission:\n")
for func_id in range(1, 9):
    print(f"Function {func_id}: {results[func_id]}")